In [1]:
import utils
import pandas as pd
import dowhy
from dowhy import CausalModel
import numpy as np
import re

c:\Users\antonio\anaconda3\envs\causal_ds_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lalonde_original, nswre_df, cps_control, psid_control, cps2_control, cps3_control, psid2_control, psid3_control = utils.load_data()
nswre_cps, nswre_cps2, nswre_cps3, nswre_psid, nswre_psid2, nswre_psid3 = utils.load_combinations(nswre_df, cps_control, cps2_control, cps3_control,  psid_control, psid2_control, psid3_control)

graph = utils.extract_CD_graph("Forced_Graph") #Loads the graph from json
#utils.draw_graph(graph) #plots the graph
combinations_dict = {"nswre_experiment": nswre_df, 
                     "nswre_cps": nswre_cps, 
                     "nswre_cps2": nswre_cps2, 
                     "nswre_cps3": nswre_cps3, 
                     "nswre_psid": nswre_psid, 
                     "nswre_psid2": nswre_psid2, 
                     "nswre_psid3": nswre_psid3}

## Identifying Estimands and Estimating the Causal Effect and Sensitivity Analysis

In [3]:
utils.causal_estimation(combinations_dict, 
                  graph, 
                  methods=["backdoor.propensity_score_weighting"], 
                  refuter_list = ["bootstrap_refuter", "data_subset_refuter"], 
                  treatment="treatment",
                  outcome="re78")

Fitting causal mechanism of node re78: 100%|██████████| 9/9 [00:00<00:00, 155.17it/s]


,backdoor,backdoor.propensity_score_weighting,bootstrap_refuterbackdoor.propensity_score_weighting,data_subset_refuterbackdoor.propensity_score_weighting,probabilistic_effect
nswre_experiment,"[married, re75, nodegree]",1563.329507,1444.996937,1529.464603,1697.604137
nswre_cps,"[married, re75, nodegree]",-8193.897814,8205.591835,8236.759492,-4426.348616
nswre_cps2,"[married, re75, nodegree]",-3276.954688,3312.878164,3247.674335,-1459.810794
nswre_cps3,"[married, re75, nodegree]",378.536121,322.993730,378.419655,1094.913850
nswre_psid,"[married, re75, nodegree]",-13065.420938,13128.128276,13084.405009,-7446.113984
nswre_psid2,"[married, re75, nodegree]",215.255470,232.337055,91.164449,-2602.788388
nswre_psid3,"[married, re75, nodegree]",2179.493380,2088.891062,2123.830808,839.822980


# Extra

## Propensity Score Estimation

In [ ]:
from sklearn.linear_model import LogisticRegression

# Assuming X contains the covariates and treatment variable, and Y contains the outcome variable
X_covariates = final_df[COVARIATES]  # Select relevant covariates
treatment = final_df['treatment']  # Treatment variable (binary)
outcome = final_df['re78']  # Outcome variable (binary or continuous)

# Fit logistic regression model to estimate propensity scores
logistic_model = LogisticRegression(max_iter=10000)
logistic_model.fit(X_covariates, treatment)

# Predict propensity scores
propensity_scores = logistic_model.predict_proba(X_covariates)[:, 1]

# Step 2: Define propensity score ranges
# Example: Define a range from 0.2 to 0.8
min_propensity = 0.2
max_propensity = 0.8

# Step 3: Filter the data
# Create a mask to filter data points within the defined propensity score range
mask = np.logical_and(propensity_scores >= min_propensity, propensity_scores <= max_propensity)

# Apply the mask to filter the data
filtered_data = X[mask]
filtered_treatment = treatment[mask]
filtered_outcome = outcome[mask]

In [ ]:
import statsmodels.api as sm

covariates = ["treatment",  "age", "education", "black" , "hispanic", "married", "nodegree", "re74"]
# Assuming X is the treatment variable and Y is the outcome variable
# data should contain the variables X and Y, along with any other relevant variables
# Perform ordinary least squares (OLS) regression
model = sm.OLS(final_df['re78'], sm.add_constant(final_df[covariates]))
results = model.fit()

# Get the estimated causal effect
causal_effect = results.params[covariates]

print("Estimated causal effect:", causal_effect)